In [ ]:
#USING VGG16
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# --- 1. Define and Print the Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Data Transformations ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224) 
])

# --- Dataset Loading ---
data_dir = 'antsbees'
image_datasets = {}
image_datasets['train'] = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
image_datasets['val'] = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)

# --- DataLoader Setup ---
# Consider adding 'num_workers' for faster data loading on CPU, especially with a GPU
train_loader = DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4 if device.type == 'cuda' else 0)
test_loader = DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=4 if device.type == 'cuda' else 0)

# --- Model Loading (VGG16) ---
model = models.vgg16(weights='IMAGENET1K_V1')

# freeze layers (same as before)
for param in model.features.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = False

# replace the final fully connected layer
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2) 

# --- 2. Move the Model to the Device ---
model.to(device)

# --- Training Setup ---
criterian = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001)

# --- Training Loop ---
for epoch in range(20):
    model.train() 
    running_loss = 0.0
    for image, label in train_loader:
        # --- 3. Move Data to the Device ---
        image, label = image.to(device), label.to(device) 
        
        optimizer.zero_grad() 
        
        output = model(image)
        loss = criterian(output, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f"Epoch {epoch+1}/{20}, Loss: {running_loss/len(train_loader):.4f}")

# --- Evaluation ---
correct = 0.0
total = 0.0
model.eval() 

with torch.no_grad():
    for image, label in test_loader:
        # --- 3. Move Data to the Device ---
        image, label = image.to(device), label.to(device)
        
        output = model(image)
        _, predicted = torch.max(output, 1) 
        correct += (predicted == label).sum().item()
        total += label.size(0)

    print(f"Test Accuracy: {(correct/total)*100:.2f}%")

Using device: cpu
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\ashmi/.cache\torch\hub\checkpoints\vgg16-397923af.pth


100.0%


Epoch 1/20, Loss: 0.4032
Epoch 2/20, Loss: 0.1888
Epoch 3/20, Loss: 0.1156
Epoch 4/20, Loss: 0.1325
Epoch 5/20, Loss: 0.1091
Epoch 6/20, Loss: 0.0784
Epoch 7/20, Loss: 0.1163
Epoch 8/20, Loss: 0.0976
Epoch 9/20, Loss: 0.1078
Epoch 10/20, Loss: 0.0769
Epoch 11/20, Loss: 0.1203
Epoch 12/20, Loss: 0.0884
Epoch 13/20, Loss: 0.0854
Epoch 14/20, Loss: 0.0686
Epoch 15/20, Loss: 0.0906
Epoch 16/20, Loss: 0.0609
Epoch 17/20, Loss: 0.0364
Epoch 18/20, Loss: 0.0819
Epoch 19/20, Loss: 0.1108
Epoch 20/20, Loss: 0.1088
Test Accuracy: 94.77%


In [1]:
##USING INCEPTIONV3
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Data Transformations ---
# InceptionV3 requires an input size of 299x299
transform = transforms.Compose([
    transforms.ToTensor(),
    # Using the standard ImageNet mean/std
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
    transforms.RandomHorizontalFlip(),
    # CHANGE: InceptionV3 requires 299x299 input
    transforms.RandomResizedCrop(299) 
])

# --- Dataset Loading ---
data_dir = 'antsbees'
image_datasets = {}
image_datasets['train'] = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
image_datasets['val'] = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)

# --- DataLoader Setup ---
# Batch size reduced further as InceptionV3 is deep, or kept at 32/64
train_loader = DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4 if device.type == 'cuda' else 0)
test_loader = DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=4 if device.type == 'cuda' else 0)

# --- Model Loading (InceptionV3) ---
# CHANGE: Set aux_logits=False for inference/simplicity, or True for standard training
# We set it to True here to match the pre-trained weights architecture, but we'll adapt the training loop.
model = models.inception_v3(weights='IMAGENET1K_V1', aux_logits=True) 

# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

# --- Replace the Final Layers ---

# 1. Replace the primary final classification layer (fc)
# The input features for InceptionV3's fc layer are 2048
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# 2. Replace the Auxiliary classification layer (AuxLogits)
# The input features for InceptionV3's AuxLogits layer are 768
num_aux_features = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(num_aux_features, 2)

# Move the model to the device
model.to(device)

# --- Training Setup ---
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(params_to_update, lr=0.001)

# --- Training Loop ---
for epoch in range(20):
    model.train() 
    running_loss = 0.0
    for image, label in train_loader:
        image, label = image.to(device), label.to(device) 
        
        optimizer.zero_grad() 
        output, aux_output = model(image) 
        
        loss_main = criterion(output, label)  
        loss_aux = criterion(aux_output, label)  
        loss = loss_main + 0.4 * loss_aux 
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f"Epoch {epoch+1}/{20}, Loss: {running_loss/len(train_loader):.4f}")

# --- Evaluation ---
correct = 0.0
total = 0.0
# IMPORTANT: Set model to evaluation mode, which automatically handles the AuxLogits
model.eval() 

with torch.no_grad():
    for image, label in test_loader:
        image, label = image.to(device), label.to(device)
        
        # CHANGE: In evaluation mode, InceptionV3 returns only the primary output
        output = model(image) 
        
        _, predicted = torch.max(output, 1) 
        correct += (predicted == label).sum().item()
        total += label.size(0)

    print(f"Test Accuracy: {(correct/total)*100:.2f}%")

Using device: cpu
Epoch 1/20, Loss: 1.0240
Epoch 2/20, Loss: 0.7777
Epoch 3/20, Loss: 0.6068
Epoch 4/20, Loss: 0.4771
Epoch 5/20, Loss: 0.4446
Epoch 6/20, Loss: 0.3744
Epoch 7/20, Loss: 0.3611
Epoch 8/20, Loss: 0.3367
Epoch 9/20, Loss: 0.3287
Epoch 10/20, Loss: 0.2950
Epoch 11/20, Loss: 0.3809
Epoch 12/20, Loss: 0.2445
Epoch 13/20, Loss: 0.2970
Epoch 14/20, Loss: 0.2812
Epoch 15/20, Loss: 0.2678
Epoch 16/20, Loss: 0.2926
Epoch 17/20, Loss: 0.2753
Epoch 18/20, Loss: 0.2484
Epoch 19/20, Loss: 0.2701
Epoch 20/20, Loss: 0.2323
Test Accuracy: 90.20%


In [2]:
#USING DenseNet121
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(128)
])

data_dir = 'antsbees'
image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform),
    'val': datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
}

train_loader = DataLoader(image_datasets['train'], batch_size=128, shuffle=True)
test_loader = DataLoader(image_datasets['val'], batch_size=128, shuffle=False)

# Load pretrained DenseNet-121
model = models.densenet121(weights='IMAGENET1K_V1')
print(model)

# Freeze pretrained layers
for param in model.parameters():
    param.requires_grad = False

# Replace the classifier for 2 classes
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 2)

criterian = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    for image, label in train_loader:
        image, label = image.to(device), label.to(device)
        output = model(image)
        loss = criterian(output, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Evaluation
correct = 0.0
total = 0.0
model.eval()
with torch.no_grad():
    for image, label in test_loader:
        image, label = image.to(device), label.to(device)
        output = model(image)
        _, predicted = torch.max(output, 1)
        correct += (predicted == label).sum().item()
        total += label.size(0)

print(f"Test Accuracy: {(correct / total) * 100:.2f}%")


Using device: cpu
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\ashmi/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth


100.0%


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [3]:
#USING MobileNetV2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(128)
])

data_dir = 'antsbees'
image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform),
    'val': datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
}

train_loader = DataLoader(image_datasets['train'], batch_size=128, shuffle=True)
test_loader = DataLoader(image_datasets['val'], batch_size=128, shuffle=False)

# Load pretrained MobileNetV2
model = models.mobilenet_v2(weights='IMAGENET1K_V1')
print(model)

# Freeze pretrained layers
for param in model.parameters():
    param.requires_grad = False

# Replace the last linear layer
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 2)

criterian = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    for image, label in train_loader:
        image, label = image.to(device), label.to(device)
        output = model(image)
        loss = criterian(output, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Evaluation
correct = 0.0
total = 0.0
model.eval()
with torch.no_grad():
    for image, label in test_loader:
        image, label = image.to(device), label.to(device)
        output = model(image)
        _, predicted = torch.max(output, 1)
        correct += (predicted == label).sum().item()
        total += label.size(0)

print(f"Test Accuracy: {(correct / total) * 100:.2f}%")


Using device: cpu
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\ashmi/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


100.0%


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=